In [33]:
import pandas as pd
import numpy as np
import sqlite3
import csv
import openpyxl
import adawat.adaat as ad
from collections import Counter

In [3]:
conn = sqlite3.connect('C:/Users/dzak/Documents/SERTIFIKAT BANGKIT 24/projek capstone/arramooz-master/arramooz-master/data/arabicdictionary.sqlite')
cursor = conn.cursor()

# Misalnya, kita mengambil kolom-kolom penting
cursor.execute("SELECT id, unvocalized, vocalized, root, normalized , triliteral, past, future, imperative, passive FROM verbs")
data = cursor.fetchall()

# Menutup koneksi database setelah selesai mengambil data
conn.close()
with open('C:/Users/dzak/Documents/SERTIFIKAT BANGKIT 24/projek capstone/arramooz-master/arabic_verbs_dataset.csv', mode='w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)

    # Tulis header kolom sesuai dengan nama kolom yang diambil
    writer.writerow(['ID', 'Unvocalized', 'Vocalized', 'Root', 'Normalized' ,'Triliteral', 'Past', 'Future', 'Imperative', 'Passive'])

    # Tulis setiap baris data ke file CSV
    for row in data:
        writer.writerow(row)

print("Data berhasil disimpan ke output/arabic_verbs_dataset.csv")

Data berhasil disimpan ke output/arabic_verbs_dataset.csv


In [4]:
conn = sqlite3.connect('C:/Users/dzak/Documents/SERTIFIKAT BANGKIT 24/projek capstone/arramooz-master/arramooz-master/data/arabicdictionary.sqlite')
cursor = conn.cursor()

# Misalnya, kita mengambil kolom-kolom penting
cursor.execute("SELECT id, unvocalized, vocalized, wordtype ,root, wazn, normalized , stamped,category, original, mankous, defined, gender, feminin,masculin,number,single,dualable,feminable,masculin_plural,feminin_plural,broken_plural,mamnou3_sarf,relative,w_suffix,hm_suffix,kal_prefix,ha_suffix,k_prefix,annex,definition,note FROM nouns")
data = cursor.fetchall()

# Menutup koneksi database setelah selesai mengambil data
conn.close()
with open('C:/Users/dzak/Documents/SERTIFIKAT BANGKIT 24/projek capstone/arramooz-master/arabic_nouns_dataset.csv', mode='w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)

    # Tulis header kolom sesuai dengan nama kolom yang diambil
    writer.writerow(['ID', 'unvocalized', 'vocalized', 'wordtype' , 'root', 'wazn', 'normalized' , 'stamped','category' , 'original', 'mankous', 'defined', 'gender', 'feminin','masculin','number','single','dualable','feminable','masculin_plural','feminin_plural','broken_plural','mamnou3_sarf','relative','w_suffix','hm_suffix','kal_prefix','ha_suffix','k_prefix','annex','definition','note'])

    # Tulis setiap baris data ke file CSV
    for row in data:
        writer.writerow(row)

print("Data berhasil disimpan ke output/arabic_nouns_dataset.csv")

Data berhasil disimpan ke output/arabic_nouns_dataset.csv


Data preprocessing dataset verbs

In [3]:
file_path = 'C:/Users/dzak/Documents/SERTIFIKAT BANGKIT 24/projek capstone/arramooz-master'  # Sesuaikan dengan path yang benar

# Membaca file CSV dan menampilkannya sebagai DataFrame
df_verbs = pd.read_csv(file_path +'/arabic_verbs_dataset.csv', encoding='utf-8')
df_verbs.head()

,ID,Unvocalized,Vocalized,Root,Normalized,Triliteral,Past,Future,Imperative,Passive
0,1,أمكن,أمْكَنَ,مكن,ءمكن,0,0,0,0,0
1,2,استحال,اِسْتَحَالَ,حيل,استحال,0,0,0,0,0
2,3,زها,زَهَا,زهو,زها,1,0,0,0,1
3,4,بئس,بِئْسَ,بأس,بءس,1,0,0,0,0
4,5,أغمى,أَغْمَى,غمي,ءغمى,0,0,0,0,1


In [4]:
# Isi nilai kosong
df_verbs.fillna('unknown', inplace=True)


In [5]:
# Ganti nama kolom jika diperlukan
df_verbs.rename(columns={
    'Unvocalized': 'unvocalized', 
    'Vocalized': 'vocalized',
    'Root': 'root',
    'Normalized': 'normalized',
    'Triliteral':'triliteral',
    'Past':'past',
    'Future':'future',
    'Imperative':'imperative',
    'Passive':'passive'
}, inplace=True)


In [5]:
df_verbs.head()

,ID,unvocalized,vocalized,root,normalized,triliteral,past,future,imperative,passive
0,1,أمكن,أمْكَنَ,مكن,ءمكن,0,0,0,0,0
1,2,استحال,اِسْتَحَالَ,حيل,استحال,0,0,0,0,0
2,3,زها,زَهَا,زهو,زها,1,0,0,0,1
3,4,بئس,بِئْسَ,بأس,بءس,1,0,0,0,0
4,5,أغمى,أَغْمَى,غمي,ءغمى,0,0,0,0,1


In [6]:
file_path = r"C:\Users\dzak\Documents\SERTIFIKAT BANGKIT 24\projek capstone\HasilGenerateUnique\arabicmorph.xlsx"

# Membaca file CSV dan menampilkannya sebagai DataFrame
df = pd.read_excel(file_path, sheet_name='Sheet1')
df.head()

,Unnamed: 0,Arabic,Buckwalter,Pattern,Root,Vector,Wazan,WazanIndex,Type of Word,Dhamir,MSD
0,0,مَظْلُوْمٌ,maZoluwomN,ma|ouwoN|,Zlm/ظلم,"29, 17.0, 7.0, 23.0, 9.0, 24.0, 8.0, 1",Iai,10,اِسِمْ مَفْعُوْلْ,هُوَ,"{""POS"": ""N"", ""NUM"": ""SG"", ""GEN"": ""MASC"", ""Verb..."
1,1,ظَالِمٌ,ZaAlimN,|aAiN|,Zlm/ظلم,"1, 17.0, 4.0, 23.0, 15.0, 24.0, 8.0, 1",Iai,10,اِسِمْ فَاعِلْ,هُوَ,"{""POS"": ""N"", ""NUM"": ""SG"", ""GEN"": ""MASC"", ""Verb..."
2,2,يَظْلِمُ,yaZolimu,ya|oiu|,Zlm/ظلم,"35, 17.0, 7.0, 23.0, 15.0, 24.0, 12.0, 1",Iai,10,فِعِلْ مُضَارِعْ,هُوَ,"{""POS"": ""V"", ""aspect"": ""IPFV"", ""tense"": ""PRS/F..."
3,3,ظَلَمَ,Zalama,|aaa|,Zlm/ظلم,"1, 17.0, 5.0, 23.0, 14.0, 24.0, 9.0, 1",Iai,10,فِعِلْ مَاضِ,هُوَ,"{""POS"": ""V"", ""aspect"": ""PFV"", ""tense"": ""PST"", ..."
4,4,مَظْلُوْمَانِ,maZoluwomaAni,ma|ouwoa|Ani,Zlm/ظلم,"29, 17.0, 7.0, 23.0, 9.0, 24.0, 9.0, 16",Iai,10,اِسِمْ مَفْعُوْلْ,هُمَا,"{""POS"": ""N"", ""NUM"": ""DU"", ""Verb Form"": ""Iai""}"


In [7]:
columns_to_drop = ['Buckwalter', 'Vector', 'Pattern', 'MSD']
df = df.drop(columns=columns_to_drop)

# Display the updated DataFrame
df.head()

,Unnamed: 0,Arabic,Root,Wazan,WazanIndex,Type of Word,Dhamir
0,0,مَظْلُوْمٌ,Zlm/ظلم,Iai,10,اِسِمْ مَفْعُوْلْ,هُوَ
1,1,ظَالِمٌ,Zlm/ظلم,Iai,10,اِسِمْ فَاعِلْ,هُوَ
2,2,يَظْلِمُ,Zlm/ظلم,Iai,10,فِعِلْ مُضَارِعْ,هُوَ
3,3,ظَلَمَ,Zlm/ظلم,Iai,10,فِعِلْ مَاضِ,هُوَ
4,4,مَظْلُوْمَانِ,Zlm/ظلم,Iai,10,اِسِمْ مَفْعُوْلْ,هُمَا


In [8]:
columns_to_drop = ['Wazan', 'WazanIndex']
df = df.drop(columns=columns_to_drop)
df.head()

,Unnamed: 0,Arabic,Root,Type of Word,Dhamir
0,0,مَظْلُوْمٌ,Zlm/ظلم,اِسِمْ مَفْعُوْلْ,هُوَ
1,1,ظَالِمٌ,Zlm/ظلم,اِسِمْ فَاعِلْ,هُوَ
2,2,يَظْلِمُ,Zlm/ظلم,فِعِلْ مُضَارِعْ,هُوَ
3,3,ظَلَمَ,Zlm/ظلم,فِعِلْ مَاضِ,هُوَ
4,4,مَظْلُوْمَانِ,Zlm/ظلم,اِسِمْ مَفْعُوْلْ,هُمَا


In [12]:
unique_values =df['Type of Word'].unique()
print("Unique values in 'Type of Word' column:", unique_values)
df['Type of Word'] = df['Type of Word'].str.strip()

Unique values in 'Type of Word' column: [nan]


AttributeError: Can only use .str accessor with string values!

In [13]:
type_mapping = {
    'فِعِلْ مَاضِ': 1,
    'فِعِلْ مُضَارِعْ': 2,
    'مَصْدَرْ': 3,
    'اِسِمْ فَاعِلْ': 4,
    'اِسِمْ مَفْعُوْلْ': 5,
    'فِعِلْ الأَمرْ': 6,
    'فِعِلْ النَهْيِ': 7
}

# Mengubah nilai pada kolom 'Type of Word'
df['Type of Word'] = df['Type of Word'].map(type_mapping)

# Tampilkan beberapa baris dari DataFrame setelah mapping
df

,Unnamed: 0,Arabic,Root,Type of Word,Dhamir
0,0,مَظْلُوْمٌ,Zlm/ظلم,NaN,هُوَ
1,1,ظَالِمٌ,Zlm/ظلم,NaN,هُوَ
2,2,يَظْلِمُ,Zlm/ظلم,NaN,هُوَ
3,3,ظَلَمَ,Zlm/ظلم,NaN,هُوَ
4,4,مَظْلُوْمَانِ,Zlm/ظلم,NaN,هُمَا
...,...,...,...,...,...
30931,30931,عَبَسْتُنَّ,Ebs/عبس,NaN,أَنْتُنَّ
30932,30932,أَعْبِسُ,Ebs/عبس,NaN,أَنَا
30933,30933,عَبَسْتُ,Ebs/عبس,NaN,أَنَا
30934,30934,نَعْبِسُ,Ebs/عبس,NaN,نَحْنُ


In [44]:
import arramooz.arabicdictionary 
mydict = arramooz.arabicdictionary.ArabicDictionary('verbs')
wordlist = [u"استقلّ", u'استقل', u"كذب"]
tmp_list = []
for word in wordlist:
    foundlist = mydict.lookup(word)
    for word_tuple in foundlist:
        word_tuple = dict(word_tuple) 
        vocalized = word_tuple['vocalized']
        tmp_list.append(dict(word_tuple))
print(tmp_list)

[{'id': 4740, 'vocalized': 'اِسْتَقَلَّ', 'unvocalized': 'استقل', 'root': 'قلل', 'normalized': 'استقل', 'stamped': 'ستقل', 'future_type': 'فتحة', 'triliteral': 0, 'transitive': 1, 'double_trans': 0, 'think_trans': 1, 'unthink_trans': 0, 'reflexive_trans': 0, 'past': 1, 'future': 1, 'imperative': 1, 'passive': 1, 'future_moode': 1, 'confirmed': 1}, {'id': 118, 'vocalized': 'كَذَّبَ', 'unvocalized': 'كذب', 'root': 'كذب', 'normalized': 'كذب', 'stamped': 'كذب', 'future_type': 'فتحة', 'triliteral': 0, 'transitive': 1, 'double_trans': 0, 'think_trans': 1, 'unthink_trans': 0, 'reflexive_trans': 0, 'past': 1, 'future': 1, 'imperative': 1, 'passive': 1, 'future_moode': 1, 'confirmed': 1}, {'id': 10194, 'vocalized': 'كَذَبَ', 'unvocalized': 'كذب', 'root': 'كذب', 'normalized': 'كذب', 'stamped': 'كذب', 'future_type': 'كسرة', 'triliteral': 1, 'transitive': 1, 'double_trans': 0, 'think_trans': 1, 'unthink_trans': 0, 'reflexive_trans': 0, 'past': 1, 'future': 1, 'imperative': 1, 'passive': 1, 'future

In [ ]:
unvocalized_words = []
with open('arramooz-master/arabic_verbs_dataset.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for i,row in enumerate(reader):
        if i == 50:
            break
        unvocalized_words.append(row['Unvocalized'])

        

In [43]:
text = u"أمكن"
lastmark = True

# Analisis morfologi menggunakan full_stemmer
result = ad.full_stemmer(text, lastmark)
print(result)

c:\Users\dzak\AppData\Local\Programs\Python\Python311\Lib\site-packages\qalsadi\cache_pickledb.py  Error: Can't Open data base  c:\Users\dzak\AppData\Local\Programs\Python\Python311\Lib\site-packages\adawat\../tmp\.qalsadiCache.pickledb
[{'word': 'أمكن', 'vocalized': 'أَمَّكُنَّ', 'unvocalized': 'أمكن', 'semivocalized': 'أَمّكُنَّ', 'tags': 'الماضي المعلوم:هو:استفهام:y:', 'affix_key': '---كن|الماضي المعلوم:هو:استفهام:y', 'stem': 'أم', 'original_tags': ('',), 'freq': 4163, 'type': 'Verb', 'original': 'أَمَّ', 'tag_regular': True, 'root': 'ءمم', 'affix': '---كن', 'action': '', 'object_type': '', 'need': '', 'tag_type': 2, 'tag_added': False, 'tag_initial': False, 'tag_transparent': False, 'tag_mamnou3': False, 'tag_break': True, 'tag_voice': 'معلوم', 'tag_mood': '', 'tag_confirmed': '', 'tag_pronoun': 'هو', 'tag_transitive': True, 'tag_person': 4, 'tag_original_number': 'مفرد', 'tag_original_gender': 'مذكر', 'tag_number': 1, 'tag_gender': 1, 'tag_tense': 'الماضي المعلوم', 'unvoriginal': 

In [54]:
df = pd.read_csv('arramooz-master/arabic_verbs_dataset.csv')
df = df.drop(columns=["Triliteral", "Past", "Future", "Imperative", "Passive"])
df.head()

,ID,Unvocalized,Vocalized,Root,Normalized
0,1,أمكن,أمْكَنَ,مكن,ءمكن
1,2,استحال,اِسْتَحَالَ,حيل,استحال
2,3,زها,زَهَا,زهو,زها
3,4,بئس,بِئْسَ,بأس,بءس
4,5,أغمى,أَغْمَى,غمي,ءغمى


In [ ]:
lastmark = True
result = []
for word in unvocalized_words:
    result.extend(ad.full_stemmer(word, lastmark))
def check_verb_type(data):
    jamid = []
    musytaq = []
    for item in data:
        word_type = item.get('type', '')
        if 'جامد' in word_type:
            jamid.append(item['word'])
        elif '' in word_type:
            musytaq.append(item['word'])
        else:
            print(f"Kata '{item['word']}' tidak dapat dikategorikan.")
    return jamid, musytaq

# Panggil fungsi untuk memeriksa tipe kata kerja

def process_words(jamid,musytaq):
    jamid_count = Counter(jamid)
    musytaq_count = Counter(musytaq)

    all_words = set(jamid + musytaq)
    final_jamid = []
    final_musytaq = []

    for word in all_words:
       if word in musytaq_count and word not in jamid_count:
           final_musytaq.append(word)
       elif word in musytaq_count and word in jamid_count:
            if musytaq_count[word] > jamid_count[word]:
                final_musytaq.append(word)
            else:
                final_jamid.append(word)
       elif word in jamid_count and word not in musytaq_count:
            final_jamid.append(word)
    return final_jamid, final_musytaq

jamid, musytaq = check_verb_type(result)
final_jamid,final_musytaq = process_words(jamid, musytaq)
print("Kata jamid yang diproses:", final_jamid)
print("Kata musytaq yang diproses:", final_musytaq)



df['jamid'] = df.apply(lambda row: 0 if row['Unvocalized'] in final_jamid else 1, axis=1)
df['musytaq'] = df.apply(lambda row: 1 if row['Unvocalized'] in final_musytaq else 0, axis=1)

df

c:\Users\dzak\AppData\Local\Programs\Python\Python311\Lib\site-packages\qalsadi\cache_pickledb.py  Error: Can't Open data base  c:\Users\dzak\AppData\Local\Programs\Python\Python311\Lib\site-packages\adawat\../tmp\.qalsadiCache.pickledb
c:\Users\dzak\AppData\Local\Programs\Python\Python311\Lib\site-packages\qalsadi\cache_pickledb.py  Error: Can't Open data base  c:\Users\dzak\AppData\Local\Programs\Python\Python311\Lib\site-packages\adawat\../tmp\.qalsadiCache.pickledb
c:\Users\dzak\AppData\Local\Programs\Python\Python311\Lib\site-packages\qalsadi\cache_pickledb.py  Error: Can't Open data base  c:\Users\dzak\AppData\Local\Programs\Python\Python311\Lib\site-packages\adawat\../tmp\.qalsadiCache.pickledb
c:\Users\dzak\AppData\Local\Programs\Python\Python311\Lib\site-packages\qalsadi\cache_pickledb.py  Error: Can't Open data base  c:\Users\dzak\AppData\Local\Programs\Python\Python311\Lib\site-packages\adawat\../tmp\.qalsadiCache.pickledb
c:\Users\dzak\AppData\Local\Programs\Python\Python31

,ID,Unvocalized,Vocalized,Root,Normalized,jamid,musytaq
0,1,أمكن,أمْكَنَ,مكن,ءمكن,1,1
1,2,استحال,اِسْتَحَالَ,حيل,استحال,1,1
2,3,زها,زَهَا,زهو,زها,1,1
3,4,بئس,بِئْسَ,بأس,بءس,0,0
4,5,أغمى,أَغْمَى,غمي,ءغمى,1,1
...,...,...,...,...,...,...,...
10632,10633,تبول,تبوّل,جذر,تبول,1,0
10633,10634,تجلط,تجلّط,جذر,تجلط,1,0
10634,10635,تمحور,تمحور,جذر,تمحور,1,0
10635,10636,تناصح,تناصح,جذر,تناصح,1,0


,ID,Unvocalized,Vocalized,Root,Normalized
0,1,أمكن,أمْكَنَ,مكن,ءمكن
1,2,استحال,اِسْتَحَالَ,حيل,استحال
2,3,زها,زَهَا,زهو,زها
3,4,بئس,بِئْسَ,بأس,بءس
4,5,أغمى,أَغْمَى,غمي,ءغمى


,ID,Unvocalized,Vocalized,Root,Normalized,jamid,musytaq
0,1,أمكن,أمْكَنَ,مكن,ءمكن,1,1
1,2,استحال,اِسْتَحَالَ,حيل,استحال,1,1
2,3,زها,زَهَا,زهو,زها,1,1
3,4,بئس,بِئْسَ,بأس,بءس,0,0
4,5,أغمى,أَغْمَى,غمي,ءغمى,1,1
...,...,...,...,...,...,...,...
10632,10633,تبول,تبوّل,جذر,تبول,1,0
10633,10634,تجلط,تجلّط,جذر,تجلط,1,0
10634,10635,تمحور,تمحور,جذر,تمحور,1,0
10635,10636,تناصح,تناصح,جذر,تناصح,1,0
